In [2]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os
import torch.nn.functional as F
import cv2
import numpy as np
import torch.utils.data as Data

blank = cv2.imread('blank_28x28.png')
b = np.array(blank,dtype=np.float32)
b = np.transpose(b,(2,1,0))
b = torch.from_numpy(b)

num_epochs = 100
learning_rate = 1e-5


from PIL import  Image,ImageDraw
import numpy as np


data1 = []
data2= []

for i in range(20000):
    img = Image.new('RGB', (28, 28), color = 'white')
    row,col,ch= np.shape(img)
    mean = 0
    var = 1
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    noisy = img + gauss
    im2 = Image.fromarray(noisy,'RGB')
    r = np.random.randint(1,101)
    draw = ImageDraw.Draw(im2)
    if r%2==0:
        draw.rectangle([9,12,12,15],fill='red',outline='red')
        data1.append(np.asarray(im2))
    else:
        draw.rectangle([21, 12, 24, 15], fill='red', outline='red')
        data1.append(np.asarray(im2))



for i in range(20000):
    img = Image.new('RGB', (28, 28), color = 'white')
    row,col,ch= np.shape(img)
    mean = 0
    var = 1
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    noisy = img + gauss
    im2 = Image.fromarray(noisy,'RGB')
    r = np.random.randint(1,101)
    draw = ImageDraw.Draw(im2)
    if r%2==0:
        draw.rectangle([21, 12, 24, 15], fill='red', outline='red')
        data2.append(np.asarray(im2))
    else:
        draw.rectangle([9,12,12,15],fill='red',outline='red')
        data2.append(np.asarray(im2))
        
        
        
y0 = np.array(data1, dtype=np.float32)
y1 = np.array(data2, dtype=np.float32)
y0 = np.transpose(y0, (0, 3, 2, 1))# This to make channels first for CNN
y1 = np.transpose(y1, (0, 3, 2, 1)) 


print(y0.shape)

x = torch.from_numpy(y0)
y = torch.from_numpy(y1)



torch_dataset = Data.TensorDataset(x,y)


loader = Data.DataLoader(

    dataset=torch_dataset,

    batch_size=250,

    shuffle=True,

    num_workers=0,

)

class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, stride=3, padding=1),  # b, 16, 10, 10
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),  # b, 16, 5, 5
            nn.Conv2d(16, 8, 3, stride=2, padding=1),  # b, 8, 3, 3
            nn.ReLU(True), # b, 8, 2, 2
            nn.Conv2d(8,4,3, stride=2, padding=1),
            nn.ReLU(True),
            nn.Conv2d(4,2,3, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1)
        )
        self.fc = nn.Linear(2, 1)

    def forward(self, x):
        x = self.encoder(x)
        x = x.view(-1,2)
        x = self.fc(x)
        return x

class auto_decoder(nn.Module):
    def __init__(self):
        super(auto_decoder,self).__init__()
        ## decoder layers ##
        ## a kernel of 2 and a stride of 2 will increase the spatial dims by 2
        self.t_conv1 = nn.ConvTranspose2d(4, 16, 3,padding=1)
        self.t_conv2 = nn.ConvTranspose2d(16, 3, 3,padding=1)
        
    def forward(self, x):
        ## decode ##
        # add transpose conv layers, with relu activation function
        x = F.relu(self.t_conv1(x))
        # output layer (with sigmoid for scaling from 0 to 1)
        x = F.sigmoid(self.t_conv2(x))
                
        return x
    

model = autoencoder()


def h_score(fx, gy):
 
    fx = fx - fx.mean(0)

    gy = gy - gy.mean(0)

    Nsamples = fx.size(0)

    covf = torch.matmul((fx.t()), fx) / Nsamples

    covg = torch.matmul((gy.t()), (gy)) / Nsamples

    h = -2 * torch.mean((fx * gy).sum(1)) + (covf * covg).sum()

    return h


criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,
                             weight_decay=1e-5)

model_1 = auto_decoder()
optimizer_1 = torch.optim.Adam(model_1.parameters(), lr=learning_rate,
                             weight_decay=1e-5)


for epoch in range(num_epochs):
    for x,y in loader:

        # ===================forward=====================
        #loss = criterion(output1, img)
        loss = h_score(model(x),model(y))
        # ===================backward====================
        optimizer.zero_grad()
        #optimizer_1.zero_grad()
        loss.backward()
        optimizer.step()
        #optimizer_1.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch+1, num_epochs, loss.item()))
    print("x: ", model(x[0].reshape(1,3,28,28)))
    print("y:", model(y[0].reshape(1,3,28,28)))

    print("b:", model(b.reshape(1,3,28,28)))



torch.save(model.state_dict(), './conv_autoencoder.pth')

(20000, 3, 28, 28)
epoch [1/100], loss:0.0182
x:  tensor([[0.1327]], grad_fn=<AddmmBackward>)
y: tensor([[-0.4849]], grad_fn=<AddmmBackward>)
b: tensor([[-0.1143]], grad_fn=<AddmmBackward>)
epoch [2/100], loss:0.0069
x:  tensor([[0.3992]], grad_fn=<AddmmBackward>)
y: tensor([[0.4155]], grad_fn=<AddmmBackward>)
b: tensor([[0.0275]], grad_fn=<AddmmBackward>)
epoch [3/100], loss:-0.0010
x:  tensor([[0.2872]], grad_fn=<AddmmBackward>)
y: tensor([[0.2829]], grad_fn=<AddmmBackward>)
b: tensor([[0.1211]], grad_fn=<AddmmBackward>)
epoch [4/100], loss:0.0021
x:  tensor([[0.4136]], grad_fn=<AddmmBackward>)
y: tensor([[0.4515]], grad_fn=<AddmmBackward>)
b: tensor([[0.1539]], grad_fn=<AddmmBackward>)
epoch [5/100], loss:-0.0024
x:  tensor([[-0.3405]], grad_fn=<AddmmBackward>)
y: tensor([[0.3133]], grad_fn=<AddmmBackward>)
b: tensor([[0.1927]], grad_fn=<AddmmBackward>)
epoch [6/100], loss:-0.0011
x:  tensor([[0.2449]], grad_fn=<AddmmBackward>)
y: tensor([[0.4156]], grad_fn=<AddmmBackward>)
b: tenso

epoch [49/100], loss:-0.0020
x:  tensor([[0.2927]], grad_fn=<AddmmBackward>)
y: tensor([[0.3625]], grad_fn=<AddmmBackward>)
b: tensor([[0.1698]], grad_fn=<AddmmBackward>)
epoch [50/100], loss:-0.0017
x:  tensor([[0.2053]], grad_fn=<AddmmBackward>)
y: tensor([[0.2631]], grad_fn=<AddmmBackward>)
b: tensor([[0.1747]], grad_fn=<AddmmBackward>)
epoch [51/100], loss:-0.0045
x:  tensor([[0.1206]], grad_fn=<AddmmBackward>)
y: tensor([[0.3006]], grad_fn=<AddmmBackward>)
b: tensor([[0.1829]], grad_fn=<AddmmBackward>)
epoch [52/100], loss:-0.0039
x:  tensor([[0.0943]], grad_fn=<AddmmBackward>)
y: tensor([[0.0260]], grad_fn=<AddmmBackward>)
b: tensor([[0.1415]], grad_fn=<AddmmBackward>)
epoch [53/100], loss:0.0004
x:  tensor([[0.4004]], grad_fn=<AddmmBackward>)
y: tensor([[0.1330]], grad_fn=<AddmmBackward>)
b: tensor([[0.1643]], grad_fn=<AddmmBackward>)
epoch [54/100], loss:0.0042
x:  tensor([[0.3149]], grad_fn=<AddmmBackward>)
y: tensor([[-0.0860]], grad_fn=<AddmmBackward>)
b: tensor([[0.1648]], 

epoch [97/100], loss:-0.0047
x:  tensor([[0.2535]], grad_fn=<AddmmBackward>)
y: tensor([[0.2645]], grad_fn=<AddmmBackward>)
b: tensor([[0.0112]], grad_fn=<AddmmBackward>)
epoch [98/100], loss:-0.0065
x:  tensor([[0.2118]], grad_fn=<AddmmBackward>)
y: tensor([[0.2793]], grad_fn=<AddmmBackward>)
b: tensor([[0.0192]], grad_fn=<AddmmBackward>)
epoch [99/100], loss:-0.0039
x:  tensor([[0.2341]], grad_fn=<AddmmBackward>)
y: tensor([[0.0405]], grad_fn=<AddmmBackward>)
b: tensor([[0.0227]], grad_fn=<AddmmBackward>)
epoch [100/100], loss:-0.0158
x:  tensor([[0.1811]], grad_fn=<AddmmBackward>)
y: tensor([[0.2582]], grad_fn=<AddmmBackward>)
b: tensor([[0.0350]], grad_fn=<AddmmBackward>)


In [8]:
for i in range(20):
    img = Image.new('RGB', (28, 28), color = 'white')
    row,col,ch= np.shape(img)
    mean = 0
    var = 1
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    noisy = img + gauss
    im2_n = Image.fromarray(noisy,'RGB')
    b = np.asarray(im2_n)
    b = np.array(b, dtype=np.float32)
    b = np.transpose(b,(2,1,0))
    b = torch.from_numpy(b)
    im2_test = Image.fromarray(noisy,'RGB')
    draw1 = ImageDraw.Draw(im2_test)
    draw1.rectangle([5, 9, 8, 12], fill='red', outline='red')
    z = np.asarray(im2_test)
    z = np.array(z, dtype=np.float32)
    z = torch.from_numpy(z)
    print("b:", model(b.reshape(1,3,28,28)))
    print("z: ", model(z.reshape(1,3,28,28)))
    print("x: ", model(x[i].reshape(1,3,28,28)))
    print("y:", model(y[i].reshape(1,3,28,28)))
    print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
    print("  ")

    

b: tensor([[-0.0180]], grad_fn=<AddmmBackward>)
z:  tensor([[-0.4685]], grad_fn=<AddmmBackward>)
x:  tensor([[0.1811]], grad_fn=<AddmmBackward>)
y: tensor([[0.2582]], grad_fn=<AddmmBackward>)
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
  
b: tensor([[0.4356]], grad_fn=<AddmmBackward>)
z:  tensor([[0.1604]], grad_fn=<AddmmBackward>)
x:  tensor([[0.2327]], grad_fn=<AddmmBackward>)
y: tensor([[0.2095]], grad_fn=<AddmmBackward>)
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
  
b: tensor([[0.1829]], grad_fn=<AddmmBackward>)
z:  tensor([[0.1764]], grad_fn=<AddmmBackward>)
x:  tensor([[0.2527]], grad_fn=<AddmmBackward>)
y: tensor([[0.2012]], grad_fn=<AddmmBackward>)
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
  
b: tensor([[0.2583]], grad_fn=<AddmmBackward>)
z:  tensor([[0.2408]], grad_fn=<AddmmBackward>)
x:  tensor([[0.3077]], grad_fn=<AddmmBackward>)
y: tensor([[0.2020]], grad_fn=<AddmmBackward>)
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

In [5]:
for i in range(20):
    img = Image.new('RGB', (28, 28), color = 'white')
    row,col,ch= np.shape(img)
    mean = 0
    var = 1
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    noisy = img + gauss
    im2 = Image.fromarray(noisy,'RGB')
    b = np.asarray(im2)
    b = np.array(b, dtype=np.float32)
    b = torch.from_numpy(b)
    print("b:", model(b.reshape(1,3,28,28)))
    print("x: ", model(x[i].reshape(1,3,28,28)))
    print("y:", model(y[i].reshape(1,3,28,28)))
    print("*****************************************************")

b: tensor([[0.2892]], grad_fn=<AddmmBackward>)
x:  tensor([[0.1811]], grad_fn=<AddmmBackward>)
y: tensor([[0.2582]], grad_fn=<AddmmBackward>)
*****************************************************
b: tensor([[0.1280]], grad_fn=<AddmmBackward>)
x:  tensor([[0.2327]], grad_fn=<AddmmBackward>)
y: tensor([[0.2095]], grad_fn=<AddmmBackward>)
*****************************************************
b: tensor([[0.0571]], grad_fn=<AddmmBackward>)
x:  tensor([[0.2527]], grad_fn=<AddmmBackward>)
y: tensor([[0.2012]], grad_fn=<AddmmBackward>)
*****************************************************
b: tensor([[0.1814]], grad_fn=<AddmmBackward>)
x:  tensor([[0.3077]], grad_fn=<AddmmBackward>)
y: tensor([[0.2020]], grad_fn=<AddmmBackward>)
*****************************************************
b: tensor([[0.1673]], grad_fn=<AddmmBackward>)
x:  tensor([[-0.1108]], grad_fn=<AddmmBackward>)
y: tensor([[0.2688]], grad_fn=<AddmmBackward>)
*****************************************************
b: tensor([[-0.1247